In [1]:
!pip install PyQt5

In [3]:
!pip install imageio

In [87]:
import matplotlib
matplotlib.use("Qt5Agg")  # Asegura el backend Qt en un script normal

import matplotlib.pyplot as plt
import roboticstoolbox as rtb
import numpy as np

# Definición del robot SCARA RRP
L1 = rtb.RevoluteDH(d=0.15, a=0.077, alpha=0)
L2 = rtb.RevoluteDH(d=-0.06, a=0.072, alpha=np.pi)
L3 = rtb.PrismaticDH(theta=0, a=0, alpha=0, qlim=[0, 0.027])

robot = rtb.DHRobot([L1, L2, L3], name="SCARA_RRP")
print(robot)
# Configuración
#q_plot = [0, -np.pi/2, 0.3]

q_plot = [0, np.pi/2, 0.015]

# Dibujar el robot en una ventana PyPlot
robot.plot(q_plot, backend='pyplot', block=False)

# Mantener la ventana abierta hasta que la cierres
plt.show(block=True)

DHRobot: SCARA_RRP, 3 joints (RRP), dynamics, standard DH parameters
┌──────┬───────┬───────┬────────┬─────────┬────────┐
│  θⱼ  │  dⱼ   │  aⱼ   │   ⍺ⱼ   │   q⁻    │   q⁺   │
├──────┼───────┼───────┼────────┼─────────┼────────┤
│  q1  │  0.15 │ 0.077 │   0.0° │ -180.0° │ 180.0° │
│  q2  │ -0.06 │ 0.072 │ 180.0° │ -180.0° │ 180.0° │
│ 0.0° │    q3 │     0 │   0.0° │     0.0 │  0.027 │
└──────┴───────┴───────┴────────┴─────────┴────────┘

┌──┬──┐
└──┴──┘



In [46]:
import matplotlib
# matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import roboticstoolbox as rtb
import numpy as np
from spatialmath import SE3

# 1. Definición del robot SCARA RRP (Igual que tu código)
L1 = rtb.RevoluteDH(d=0.15, a=0.077, alpha=0)
L2 = rtb.RevoluteDH(d=-0.06, a=0.072, alpha=np.pi)
L3 = rtb.PrismaticDH(theta=0, a=0, alpha=0, qlim=[0, 0.027])

robot = rtb.DHRobot([L1, L2, L3], name="SCARA_RRP")

# ---------------------------------------------------------
# 2. Definir el Objetivo (Target) en el Espacio Cartesiano
# ---------------------------------------------------------
# Queremos que el robot vaya a: X=0.6, Y=0.2, Z=0.4
target_pos = SE3(0.1, 0.00, 0.01)

print(f"Objetivo deseado:\n{target_pos}")
# ---------------------------------------------------------
# 3. Calcular Cinemática Inversa
# ---------------------------------------------------------
# Usamos el solucionador numérico Levenberg-Marquardt (ikine_LM).
# mask=[1, 1, 1, 0, 0, 0] significa:
# Importa: [X, Y, Z,  Rx, Ry, Rz] -> Solo los 3 primeros (Posición)
solucion = robot.ikine_LM(target_pos, mask=[1, 1, 1, 0, 0, 0])
# ---------------------------------------------------------
# 4. Verificar y Graficar
# ---------------------------------------------------------
if solucion.success:
    q_sol = solucion.q
    print("\n¡Solución encontrada!")
    print(f"q1 (Revolute) : {q_sol[0]:.4f} rad")
    print(f"q2 (Revolute) : {q_sol[1]:.4f} rad")
    print(f"d3 (Prismatic): {q_sol[2]:.4f} m")
    
    # Verificar dónde terminó realmente el robot usando Cinemática Directa
    T_result = robot.fkine(q_sol)
    print(f"\nPosición alcanzada real:\n{T_result}")
    
    # Graficar
    robot.plot(q_sol, backend='pyplot', block=False)
else:
    print("\nNo se encontró una solución (el punto podría estar fuera del alcance).")

Objetivo deseado:
   1         0         0         0.1       
   0         1         0         0         
   0         0         1         0.01      
   0         0         0         1         


¡Solución encontrada!
q1 (Revolute) : -0.7986 rad
q2 (Revolute) : 1.6713 rad
d3 (Prismatic): 0.0800 m

Posición alcanzada real:
   0.6427    0.7661    0         0.1       
   0.7661   -0.6427    0        -6.238e-09  
   0         0        -1         0.01      
   0         0         0         1         



In [81]:
import matplotlib.pyplot as plt
import roboticstoolbox as rtb
import numpy as np
from spatialmath import SE3
import imageio
import os

# ---------------------------------------------------------
# 1. DEFINICIÓN DEL ROBOT SCARA (PRISMÁTICO HACIA ABAJO)
# ---------------------------------------------------------
L1 = rtb.RevoluteDH(d=0.15, a=0.077, alpha=0)
L2 = rtb.RevoluteDH(d=-0.06, a=0.072, alpha=np.pi)
L3 = rtb.PrismaticDH(theta=0, a=0, alpha=0, qlim=[0, 0.027])
# alpha = pi  →  ¡invierte el eje Z! (el prismatico baja con valores positivos)

robot = rtb.DHRobot([L1, L2, L3], name="SCARA_REAL")

# ---------------------------------------------------------
# 2. TRAYECTORIA DENTRO DEL ALCANCE REAL
# ---------------------------------------------------------
# Distancia máxima horizontal = 0.077 + 0.072 = 0.149 m

T_start = SE3(0.10, 0.00, 0.06)       # punto inicial dentro del volumen real
T_end   = SE3(0.12, 0.06, 0.035)      # baja 27 mm (0.027)

steps = 60
trajectory = rtb.ctraj(T_start, T_end, steps)

# ---------------------------------------------------------
# 3. ENTORNO VISUAL
# ---------------------------------------------------------
output_folder = "scara_frames_gif"
os.makedirs(output_folder, exist_ok=True)

q_actual = [0, 0, 0]  # configuración inicial

env = robot.plot(q_actual, backend='pyplot', block=False)
env.ax.set_xlim([-0.05, 0.20])
env.ax.set_ylim([-0.10, 0.15])
env.ax.set_zlim([0.10, 0.22])

gif_frames = []
ik_mask = [1, 1, 1, 0, 0, 0]

print(f"Calculando {steps} pasos...")

# ---------------------------------------------------------
# 4. BUCLE PRINCIPAL (MOVIMIENTO PROGRESIVO REAL)
# ---------------------------------------------------------
for i, T_target in enumerate(trajectory):

    # 🚀 Usamos la solución anterior como punto de partida
    solucion = robot.ikine_LM(T_target, q0=q_actual, mask=ik_mask)

    if solucion.success:
        q_actual = solucion.q
        robot.q = q_actual
        env.step()

        # Guardar frame
        filename = os.path.join(output_folder, f"frame_{i:03d}.png")
        env.fig.savefig(filename)
        gif_frames.append(imageio.v2.imread(filename))

        print(f"Paso {i+1}/{steps}", end='\r')

    else:
        print(f"\n⚠️ IK falló en paso {i}: objetivo fuera del alcance")

print("\nGenerando GIF...")

# ---------------------------------------------------------
# 5. GENERAR GIF
# ---------------------------------------------------------
gif_name = "scara_real_movimiento.gif"
if gif_frames:
    imageio.mimsave(gif_name, gif_frames, duration=0.05, loop=0)
    print(f"GIF creado -> {gif_name}")
else:
    print("No se generaron frames.")

plt.close('all')


Calculando 60 pasos...
Paso 60/60
Generando GIF...
GIF creado -> scara_real_movimiento.gif
